<a href="https://colab.research.google.com/github/priyadharshini13/data_science/blob/master/sherlock3rd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
from ast import literal_eval
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
sys.path.append("..")

In [217]:
!unzip src

Archive:  src.zip
replace src/deploy/classes_retrain_minimal_sample.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
from src.features.build_features import build_features
from src.features.build_features import _get_data
from src.deploy.train_sherlock import train_sherlock
from src.deploy.predict_sherlock import predict_sherlock


In [ ]:
_get_data()


3.6 GiB Done.
Unzipping...Done.
Data was downloaded.


In [4]:
train_vals = pd.read_parquet('/data/data/raw/train_values.parquet')


In [5]:
train_vals['values'][0:3].values

array(["['Global', 'United States', 'Australia']",
       "['Fiction, Adult - Non-Floating', 'Fiction, Adult', 'Fiction, Adult', 'Fiction, Adult', 'Fiction, Adult']",
       "['', '', 'University of Puerto Rico - Rio Piedras', 'University of Puerto Rico - Rio Piedras', 'University of Puerto Rico - Rio Piedras', 'University of Puerto Rico-Rio Piedras', 'University of Puerto Rico-Rio Piedras', 'University of Puerto Rico-Rio Piedras', 'University of Puerto Rico-Rio Piedras', 'University of Puerto Rico-Rio Piedras', 'University of Puerto Rico-Rio Piedras', 'PARK PLACE DEALERSHIPS', 'PARK PLACE DEALERSHIPS', 'PARK PLACE DEALERSHIPS', 'PARK PLACE DEALERSHIPS', 'Carolina Tri', 'Carolina Tri', 'Parck Place Dealerships/ Sun a', 'Parck Place Dealerships/ Sun a', 'Parck Place Dealerships/ Sun a', 'Park Place Dealerships', 'Park Place Dealerships', 'Dallas Racing', 'PARK PLACE DEALERSHIPS', 'PARK PLACE DEALERSHIPS', 'PARK PLACE DEALERSHIPS']"],
      dtype=object)

In [6]:
train_labs = pd.read_parquet('/data/data/raw/train_labels.parquet')
train_labs
# train_labs['type'==][1]

,type
55030,area
167000,collection
638282,team Name
232298,credit
316158,gender
...,...
344035,industry
167866,collection
256742,description
253456,depth


In [7]:
#Validation data
val_vals = pd.read_parquet('/data/data/raw/val_values.parquet')
val_labs = pd.read_parquet('/data/data/raw/val_labels.parquet')

In [8]:
val_vals

,values
217858,"['Shelby', 'Knox', 'Washington', 'Davidson', '..."
166650,"['CDs, Adult Musical', 'CDs, Adult Musical', '..."
27611,"[20.42, 23.17, 24.5, 24.0, 21.92, 28.08, 25.5,..."
370305,"['Nick Scholfield', 'R Johnson', 'P J Brennan'..."
45377,"['', 'Tres Hombres (Expanded & Remastered)', '..."
...,...
62381,"['Castle Gorge', 'Dome', 'Grootkloof', 'Mhlaba..."
13168,"['150 Trumbull St Fl 5 Hartford, CT 06103-2446..."
276934,"['Last 7 Games', 'Last 15 Games', 'Last 30 Gam..."
118627,"['Fr.', 'Sr.', 'Fr.', 'Fr.', 'So.', 'Jr.', 'So..."


In [ ]:
val_labs

,type
217858,county
166650,collection
27611,age
370305,jockey
45377,album
...,...
62381,area
13168,address
276934,duration
118627,class


In [9]:
#test data
test_vals = pd.read_parquet('/data/data/raw/test_values.parquet')
test_labs = pd.read_parquet('/data/data/raw/test_labels.parquet')

In [ ]:
test_vals

,values
20368,"['Central Missouri', 'unattached', 'unattached..."
664102,"[95, 100, 95, 89, 84, 91, 88, 94, 75, 78, 90, ..."
366813,"['Katie Crews', 'Christian Hiraldo', 'Alex Est..."
530567,"['Christian', 'Non-Christian', 'Unreported', '..."
176253,"['AAF-McQuay Canada Inc.', 'AAF-McQuay Canada ..."
...,...
205688,"['Madagascar', 'Malawi', 'Gambia, The', 'Brazi..."
537568,"['--', 'W 65-3', 'W 52-15', 'W 45-23', 'W 41-1..."
209598,"['Canada', 'France', 'Germany', 'Japan', 'UK',..."
590173,"['Delaware', 'District of Columbia', 'Hawaii',..."


In [ ]:
test_labs

,type
20368,affiliation
664102,weight
366813,jockey
530567,religion
176253,company
...,...
205688,country
537568,result
209598,country
590173,state


In [10]:
len(val_labs)

137353

In [ ]:
test_labs.nunique()

type    78
dtype: int64

In [ ]:
train_vals = train_vals['values'].apply(literal_eval)


KeyboardInterrupt: ignored

In [ ]:
val_vals = val_vals['values'].apply(literal_eval)


In [ ]:
test_vals = test_vals['values'].apply(literal_eval)

In [ ]:
train_vals.head()


,values
55030,"['Global', 'United States', 'Australia']"
167000,"['Fiction, Adult - Non-Floating', 'Fiction, Ad..."
638282,"['', '', 'University of Puerto Rico - Rio Pied..."
232298,"['Laughology', 'MTV', 'With Intent to Kill', '..."
316158,"['Mare', 'Gelding', 'Gelding', 'Gelding', 'Gel..."


In [233]:
def prepare_feature_extraction():
    
    word_embedding_file = 'src/features/glove.6B.50d.txt'
    paragraph_vector_file = 'src/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy'
    
    print('Preparing feature extraction by downloading 3 files: \n {} and \n {}.'.format(word_embedding_file,
                                                                                         paragraph_vector_file))

    if not os.path.exists(word_embedding_file):
        print('Downloading GloVe word embedding vectors.')
        file_name = word_embedding_file
        gd.download_file_from_google_drive(file_id='1kayd5oNRQm8-NCvA8pIrtezbQ-B1_Vmk',
                                           dest_path=file_name,
                                           unzip=False,
                                           showsize=True)

    print('GloVe word embedding vectors were downloaded.')

    if not os.path.exists(paragraph_vector_file):
        print('Downloading pretrained paragraph vectors.')
        file_name = paragraph_vector_file
        gd.download_file_from_google_drive(file_id='1vdyGJ4aB71FCaNqJKYX387eVufcH4SAu',
                                           dest_path=file_name,
                                           unzip=False,
                                           showsize=True)
        
    print('Trained paragraph vector model was downloaded.')

# X_train = build_features(train_vals)


In [266]:
import numpy as np

from scipy import stats
from collections import OrderedDict
from smart_open import open

# Input: a single column in the form of a pandas series
# Output: ordered dictionary holding word embedding features
def extract_word_embeddings_features(values):

    num_embeddings = 50
    f = OrderedDict()
    embeddings = []
    # word_vectors_f = []
    # word_vectors_f = open('src/features/glove.6B.50d.txt')
    # with open('src/features/glove.6B.50d.txt') as fin:
      #  word_vectors_f = fin.read()
    # print(len(word_vectors_f))
    # print(word_vectors_f[0:20] )
    # print(type(word_vectors_f))

    # with open('src/features/glove.6B.50d.txt','r',encoding='utf-8') as f:
    #   word_vectors_f = f.read()
      # print()
    # word_vectors_f = open('src/features/glove.6B.50d.txt', encoding='utf-8')
    word_to_embedding = {}

    # for w in word_vectors_f:
    i = 0
    with open('src/features/glove.6B.50d.txt') as fin:
        
        w = fin.read()
        if i < 2:
          print('------',w)
        w = w.strip('\n')
        if i < 2:
          print('*******',w)
        i = i+1
        term, vector = w.strip().split(' ', 1)
        # vector = vector.strip('\n')
        vector = vector.strip()
        vector = np.array(vector.split(' '))
        vector = vector.astype(np.float)

        word_to_embedding[term] = vector

    
    values = values.dropna()

    for v in values:

        v = str(v).lower()

        if v in word_to_embedding:
            embeddings.append(word_to_embedding.get(v))
        else:
            words = v.split(' ')
            embeddings_to_all_words = []

            for w in words:
                if w in word_to_embedding:
                    embeddings_to_all_words.append(word_to_embedding.get(w))
            if embeddings_to_all_words:
                mean_of_word_embeddings = np.nanmean(embeddings_to_all_words, axis=0)
                embeddings.append(mean_of_word_embeddings)

    if len(embeddings) == 0:
        for i in range(num_embeddings): f['word_embedding_avg_{}'.format(i)] = np.nan
        for i in range(num_embeddings): f['word_embedding_std_{}'.format(i)] = np.nan
        for i in range(num_embeddings): f['word_embedding_med_{}'.format(i)] = np.nan
        for i in range(num_embeddings): f['word_embedding_mode_{}'.format(i)] = np.nan

        f['word_embedding_feature'] = 0

        return f

    else:
        mean_embeddings = np.nanmean(embeddings, axis=0)
        med_embeddings = np.nanmedian(embeddings, axis=0)
        std_embeddings = np.nanstd(embeddings, axis=0)
        mode_embeddings = stats.mode(embeddings, axis=0, nan_policy='omit')[0].flatten()

        for i, e in enumerate(mean_embeddings): f['word_embedding_avg_{}'.format(i)] = e
        for i, e in enumerate(std_embeddings): f['word_embedding_std_{}'.format(i)] = e
        for i, e in enumerate(med_embeddings): f['word_embedding_med_{}'.format(i)] = e
        for i, e in enumerate(mode_embeddings): f['word_embedding_mode_{}'.format(i)] = e

        f['word_embedding_feature'] = 1

        return f


In [238]:
def _build_features(data):

    _get_data()
    prepare_feature_extraction()

    features_list = []
    df_par = pd.DataFrame()
    n_samples = 1000
    vec_dim = 400
    i = 0
    print(len(data))
    for raw_sample in data:
        if i < 2:
          print(raw_sample)
        i = i + 1
        if i % 100 == 0:
            print('Extracting features for data column ', i)

        n_values = len(raw_sample)

        if n_samples > n_values:
            n_samples = n_values
        random.seed(13)
        raw_sample = pd.Series(random.choices(raw_sample, k=n_samples)).astype(str)
        print('Reachd here')
        f = OrderedDict(list(extract_bag_of_characters_features(raw_sample).items()) +
                        list(extract_word_embeddings_features(raw_sample).items()) +
                        list(extract_bag_of_words_features(raw_sample, n_values).items()))
        features_list.append(f)
        print('Reachd features_list.append')

        df_par = df_par.append(infer_paragraph_embeddings_features(raw_sample, vec_dim))

    return pd.concat([pd.DataFrame(features_list).reset_index(drop=True),
                      df_par.reset_index(drop=True)],
                     axis=1, sort=False)


In [227]:
import random
import os
import pandas as pd

from collections import OrderedDict
from google_drive_downloader import GoogleDriveDownloader as gd
from src.features.bag_of_characters import extract_bag_of_characters_features
from src.features.bag_of_words import extract_bag_of_words_features
# from src.features.word_embeddings import extract_word_embeddings_features
# from src.features.paragraph_vectors import infer_paragraph_embeddings_features




AttributeError: ignored

In [228]:
import pandas as pd
import random

from gensim.models.doc2vec import Doc2Vec, TaggedDocument


# Input: a collection of columns stored in a dataframe column 'values'
# Output: tagged columns.
# Only needed for training.
def tagcol_paragraph_embeddings_features(train_data):
    
    # Expects a dataframe with a 'values' column
    train_data_values = train_data['values']
    random.seed(13)
    columns = [TaggedDocument(random.sample(col, min(1000, len(col))), [i]) for i, col in enumerate(train_data_values.values)]
    
    return columns


# Input: returned tagged document collection from tagcol_paragraph_embeddings_features
# Output: a stored retrained model
# Only needed for training.
def train_paragraph_embeddings_features(columns, dim):

    # Train Doc2Vec model
    model = Doc2Vec(columns, dm=0, negative=3, workers=8, vector_size=dim, epochs=20, min_count=2, seed=13)

    # Save trained model
    model_file = 'src/features/par_vec_retrained_{}.pkl'.format(dim)
    model.save(model_file)
    model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
    

# Input: a single column in the form of a pandas Series.
# Output: ordered dictionary holding paragraph vector features
def infer_paragraph_embeddings_features(data, dim):

    # Load pretrained paragraph vector model
    model = Doc2Vec.load('src/features/par_vec_trained_{}.pkl'.format(dim))

    f = pd.DataFrame()

    if len(data) > 1000:
        random.seed(13)
        vec = random.sample(data, 1000)
    else:
        vec = data

    # Infer paragraph vector for data sample
    f = f.append(pd.Series(model.infer_vector(vec, steps=20,
                                              alpha=0.025)), ignore_index=True)

    col_names = []
    for i, col in enumerate(f):
        col_names.append('par_vec_{}'.format(i))
        
    f.columns = col_names

    return f


In [1]:
X_train = _build_features(train_vals)

NameError: ignored

In [265]:
vector = '0.78581\n'
v = vector.strip('\n')
v

'0.78581'

In [229]:
X_train

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,n_[1]-agg-any,n_[1]-agg-all,n_[1]-agg-mean,n_[1]-agg-var,n_[1]-agg-min,n_[1]-agg-max,n_[1]-agg-median,n_[1]-agg-sum,n_[1]-agg-kurtosis,n_[1]-agg-skewness,n_[2]-agg-any,n_[2]-agg-all,n_[2]-agg-mean,n_[2]-agg-var,n_[2]-agg-min,n_[2]-agg-max,n_[2]-agg-median,n_[2]-agg-sum,n_[2]-agg-kurtosis,n_[2]-agg-skewness,n_[3]-agg-any,n_[3]-agg-all,n_[3]-agg-mean,n_[3]-agg-var,n_[3]-agg-min,n_[3]-agg-max,n_[3]-agg-median,n_[3]-agg-sum,n_[3]-agg-kurtosis,n_[3]-agg-skewness,...,par_vec_360,par_vec_361,par_vec_362,par_vec_363,par_vec_364,par_vec_365,par_vec_366,par_vec_367,par_vec_368,par_vec_369,par_vec_370,par_vec_371,par_vec_372,par_vec_373,par_vec_374,par_vec_375,par_vec_376,par_vec_377,par_vec_378,par_vec_379,par_vec_380,par_vec_381,par_vec_382,par_vec_383,par_vec_384,par_vec_385,par_vec_386,par_vec_387,par_vec_388,par_vec_389,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,False,False,0.0,0.0,0,0,0.0,0,-3.0,0.0,False,False,0.0,0.0,0,0,0.0,0,-3.0,0.0,False,False,0.0,0.0,0,0,0.0,0,-3.0,0.0,False,False,0.0,0.0,0,0,0.0,0,-3.0,0.0,...,0.017042,0.038366,-0.034419,0.017222,-0.016056,0.024635,0.016342,-0.023305,0.028304,-0.047105,-0.046572,0.034086,-0.044176,0.039277,-0.047624,-0.003166,-0.028353,-0.016154,-0.043296,-0.04092,-0.001659,0.004161,-0.008969,0.009182,0.065081,0.022257,0.001381,0.017444,0.017037,-0.012766,-0.035539,-0.020606,-0.012779,0.035268,-0.01455,-0.028219,0.03196,-0.001547,0.009776,-0.031648


In [ ]:
X_train.to_parquet('X_train_13.parquet')


In [ ]:
X_train.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,n_[1]-agg-any,n_[1]-agg-all,n_[1]-agg-mean,n_[1]-agg-var,n_[1]-agg-min,n_[1]-agg-max,n_[1]-agg-median,n_[1]-agg-sum,n_[1]-agg-kurtosis,n_[1]-agg-skewness,n_[2]-agg-any,n_[2]-agg-all,n_[2]-agg-mean,n_[2]-agg-var,n_[2]-agg-min,n_[2]-agg-max,n_[2]-agg-median,n_[2]-agg-sum,n_[2]-agg-kurtosis,n_[2]-agg-skewness,n_[3]-agg-any,n_[3]-agg-all,n_[3]-agg-mean,n_[3]-agg-var,n_[3]-agg-min,n_[3]-agg-max,n_[3]-agg-median,n_[3]-agg-sum,n_[3]-agg-kurtosis,n_[3]-agg-skewness,...,par_vec_360,par_vec_361,par_vec_362,par_vec_363,par_vec_364,par_vec_365,par_vec_366,par_vec_367,par_vec_368,par_vec_369,par_vec_370,par_vec_371,par_vec_372,par_vec_373,par_vec_374,par_vec_375,par_vec_376,par_vec_377,par_vec_378,par_vec_379,par_vec_380,par_vec_381,par_vec_382,par_vec_383,par_vec_384,par_vec_385,par_vec_386,par_vec_387,par_vec_388,par_vec_389,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,False,False,0.0,0.0,0,0,0.0,0,-3.0,0.0,False,False,0.0,0.0,0,0,0.0,0,-3.0,0.0,False,False,0.0,0.0,0,0,0.0,0,-3.0,0.0,False,False,0.0,0.0,0,0,0.0,0,-3.0,0.0,...,0.017042,0.038366,-0.034419,0.017222,-0.016056,0.024635,0.016342,-0.023305,0.028304,-0.047105,-0.046572,0.034086,-0.044176,0.039277,-0.047624,-0.003166,-0.028353,-0.016154,-0.043296,-0.04092,-0.001659,0.004161,-0.008969,0.009182,0.065081,0.022257,0.001381,0.017444,0.017037,-0.012766,-0.035539,-0.020606,-0.012779,0.035268,-0.01455,-0.028219,0.03196,-0.001547,0.009776,-0.031648


In [ ]:
y_train = train_labs.values.flatten()


In [ ]:
y_train

array(['area', 'collection', 'team Name', ..., 'description', 'depth',
       'product'], dtype=object)

In [ ]:
y_train

array(['area', 'collection', 'team Name', ..., 'description', 'depth',
       'product'], dtype=object)

In [ ]:

X_val = _build_features(val_labs)
X_test = _build_features(test_labs)
y_val = val_labs.values.flatten()
y_test = test_labs.values.flatten()

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
pd.DataFrame(X_train.mean()).transpose().to_csv('train_column_means.csv')


In [ ]:
train_columns_means = pd.read_csv('train_column_means.csv', index_col=0)


In [ ]:
X_train.fillna(train_columns_means.iloc[0], inplace=True)
X_val.fillna(train_columns_means.iloc[0], inplace=True)
X_test.fillna(train_columns_means.iloc[0], inplace=True)


In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import model_from_json
from tensorflow.keras.callbacks import EarlyStopping
import json


SEED = 13


def _prepare_feature_cols():
    feature_cols_dict = {}
    for feature_set in ['char', 'word', 'par', 'rest']:
        feature_cols_dict[feature_set] = pd.read_csv('src/features/feature_column_identifiers/{}_col.tsv'.format(feature_set),
                                                     sep='\t', index_col=0, header=None).values.flatten()
    return feature_cols_dict


def _prepare_label_vectors(y_train, y_val, nn_id):

    # Prepare categorical label encoder
    encoder = LabelEncoder()
    encoder.fit(y_train)
    np.save('src/deploy/classes_retrian_sherlock.npy', encoder.classes_)

    # Convert train labels
    y_train_int = encoder.transform(y_train)
    y_train_cat = tf.keras.utils.to_categorical(y_train_int)

    # Convert val labels
    y_val_int = encoder.transform(y_val)
    y_val_cat = tf.keras.utils.to_categorical(y_val_int)

    return y_train_cat, y_val_cat


def _prepare_sherlock_model(nn_id):

    lr = 0.0001
    callbacks = [EarlyStopping(monitor='val_loss', patience=5)]
    with open('src/models/retrain_minimal_sample_model.json') as f:
      data = json.load(f)
      print(data)
      sherlock = model_from_json(file.read())

    # file = open('src/models/retrain_minimal_sample_model.json', 'r')
    # print(file.read())
    # sherlock = model_from_json(file.read())
    # file.close()

    sherlock.compile(optimizer=tf.keras.optimizers.Adam(lr=lr),
                     loss='categorical_crossentropy',
                     metrics=['categorical_accuracy'])

    return sherlock, callbacks


def _save_retrained_sherlock_model(sherlock, nn_id):

    model_json = sherlock.to_json()
    with open('src/models/{}_model.json'.format(nn_id), 'w') as json:
        json.write(model_json)

    sherlock.save_weights('src/models/{}_weights.h5'.format(nn_id))


# Input: X_train and X_val dataframes as returned by the build_features method,
#        y_train and y_val arrays of string labels,
#        nn_id indicating whether to take a retrained model or sherlock
# Output: Stored retrained model
def train_sherlock(X_train, y_train, X_val, y_val, nn_id):

    y_train_cat, y_val_cat = _prepare_label_vectors(y_train, y_val, nn_id)
    feature_cols = _prepare_feature_cols()
    sherlock, callbacks = _prepare_sherlock_model(nn_id)

    print('Successfully loaded and compiled model, now fitting model on data.')

    sherlock.fit(X_train.values,
                 y_train_cat,
                 validation_data=(X_val.values,
                                  y_val_cat),
                 callbacks=callbacks, epochs=100, batch_size=256)

    _save_retrained_sherlock_model(sherlock, nn_id)

    print('Retrained Sherlock.')


In [ ]:
type(X_train)
X_train

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,n_[1]-agg-any,n_[1]-agg-all,n_[1]-agg-mean,n_[1]-agg-var,n_[1]-agg-min,n_[1]-agg-max,n_[1]-agg-median,n_[1]-agg-sum,n_[1]-agg-kurtosis,n_[1]-agg-skewness,n_[2]-agg-any,n_[2]-agg-all,n_[2]-agg-mean,n_[2]-agg-var,n_[2]-agg-min,n_[2]-agg-max,n_[2]-agg-median,n_[2]-agg-sum,n_[2]-agg-kurtosis,n_[2]-agg-skewness,n_[3]-agg-any,n_[3]-agg-all,n_[3]-agg-mean,n_[3]-agg-var,n_[3]-agg-min,n_[3]-agg-max,n_[3]-agg-median,n_[3]-agg-sum,n_[3]-agg-kurtosis,n_[3]-agg-skewness,...,par_vec_360,par_vec_361,par_vec_362,par_vec_363,par_vec_364,par_vec_365,par_vec_366,par_vec_367,par_vec_368,par_vec_369,par_vec_370,par_vec_371,par_vec_372,par_vec_373,par_vec_374,par_vec_375,par_vec_376,par_vec_377,par_vec_378,par_vec_379,par_vec_380,par_vec_381,par_vec_382,par_vec_383,par_vec_384,par_vec_385,par_vec_386,par_vec_387,par_vec_388,par_vec_389,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,0.017042,0.038366,-0.034419,0.017222,-0.016056,0.024635,0.016342,-0.023305,0.028304,-0.047105,-0.046572,0.034086,-0.044176,0.039277,-0.047624,-0.003166,-0.028353,-0.016154,-0.043296,-0.04092,-0.001659,0.004161,-0.008969,0.009182,0.065081,0.022257,0.001381,0.017444,0.017037,-0.012766,-0.035539,-0.020606,-0.012779,0.035268,-0.01455,-0.028219,0.03196,-0.001547,0.009776,-0.031648


In [ ]:
y_train_cat, y_val_cat = _prepare_label_vectors(y_train, y_val, 'retrain sherlock')


In [ ]:
feature_cols = _prepare_feature_cols()
feature_cols


In [ ]:
# X_train[feature_cols['char']].values
# X_train[feature_cols['word']].values
# X_train[feature_cols['par']].values
# X_train[feature_cols['rest']].values
y_train_cat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
# X_val[feature_cols['char']].values
# X_val[feature_cols['word']].values
# X_val[feature_cols['par']].values
# X_val[feature_cols['rest']].values
y_val_cat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
X_train

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,n_[1]-agg-any,n_[1]-agg-all,n_[1]-agg-mean,n_[1]-agg-var,n_[1]-agg-min,n_[1]-agg-max,n_[1]-agg-median,n_[1]-agg-sum,n_[1]-agg-kurtosis,n_[1]-agg-skewness,n_[2]-agg-any,n_[2]-agg-all,n_[2]-agg-mean,n_[2]-agg-var,n_[2]-agg-min,n_[2]-agg-max,n_[2]-agg-median,n_[2]-agg-sum,n_[2]-agg-kurtosis,n_[2]-agg-skewness,n_[3]-agg-any,n_[3]-agg-all,n_[3]-agg-mean,n_[3]-agg-var,n_[3]-agg-min,n_[3]-agg-max,n_[3]-agg-median,n_[3]-agg-sum,n_[3]-agg-kurtosis,n_[3]-agg-skewness,...,par_vec_360,par_vec_361,par_vec_362,par_vec_363,par_vec_364,par_vec_365,par_vec_366,par_vec_367,par_vec_368,par_vec_369,par_vec_370,par_vec_371,par_vec_372,par_vec_373,par_vec_374,par_vec_375,par_vec_376,par_vec_377,par_vec_378,par_vec_379,par_vec_380,par_vec_381,par_vec_382,par_vec_383,par_vec_384,par_vec_385,par_vec_386,par_vec_387,par_vec_388,par_vec_389,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,0.017042,0.038366,-0.034419,0.017222,-0.016056,0.024635,0.016342,-0.023305,0.028304,-0.047105,-0.046572,0.034086,-0.044176,0.039277,-0.047624,-0.003166,-0.028353,-0.016154,-0.043296,-0.04092,-0.001659,0.004161,-0.008969,0.009182,0.065081,0.022257,0.001381,0.017444,0.017037,-0.012766,-0.035539,-0.020606,-0.012779,0.035268,-0.01455,-0.028219,0.03196,-0.001547,0.009776,-0.031648


In [ ]:
# X_train_val = X_train.values.astype(float)
# X_train.values
# pd.to_numeric(X_train.values[0])
X_train = X_train.apply(pd.to_numeric).astype(float)
X_val = X_val.apply(pd.to_numeric).astype(float)
X_train
X_val
# X_val = X_val.values.astype(int)
# y_val
# X_train = pd.to_numeric(X_train.values, downcast="float")
# X_train

AttributeError: ignored

In [ ]:
train_sherlock(X_train, y_train, X_val, y_val, nn_id='retrain_sherlock');
print('Trained and saved new model.')

{'class_name': 'Model', 'config': {'name': 'model', 'layers': [{'name': 'input_1', 'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, 960], 'dtype': 'float32', 'sparse': False, 'name': 'input_1'}, 'inbound_nodes': []}, {'name': 'input_2', 'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, 201], 'dtype': 'float32', 'sparse': False, 'name': 'input_2'}, 'inbound_nodes': []}, {'name': 'input_3', 'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, 400], 'dtype': 'float32', 'sparse': False, 'name': 'input_3'}, 'inbound_nodes': []}, {'name': 'batch_normalization', 'class_name': 'BatchNormalizationV1', 'config': {'name': 'batch_normalization', 'trainable': True, 'dtype': 'float32', 'axis': [1], 'momentum': 0.99, 'epsilon': 0.001, 'center': True, 'scale': True, 'beta_initializer': {'class_name': 'Zeros', 'config': {'dtype': 'float32'}}, 'gamma_initializer': {'class_name': 'Ones', 'config': {'dtype': 'float32'}}, 'moving_mean_initializer': {'class_n

TypeError: ignored